In [2]:
import os, glob
import numpy as np
import cv2
from skimage import io, exposure, morphology, filters, measure, util

def read_mias_image(path):
    # Lê .pgm em float32 [0,1]
    img = io.imread(path).astype(np.float32)
    if img.max() > 1.0:
        img = img / 255.0
    return img

def breast_silhouette(img):
    # paper: equalização do histograma + operações morfológicas → silhueta binária
    # :contentReference[oaicite:29]{index=29}
    eq = exposure.equalize_hist(img)  # equalização global para facilitar limiarização
    thresh = filters.threshold_otsu(eq)
    mask = (eq > thresh).astype(np.uint8)
    # manter o maior componente conexo (a mama)
    labels = measure.label(mask, connectivity=2)
    if labels.max() == 0:
        return mask
    largest = 1 + np.argmax(np.bincount(labels.flat)[1:])
    mask = (labels == largest).astype(np.uint8)
    # morfologia para suavizar
    mask = morphology.binary_closing(mask, morphology.disk(5)).astype(np.uint8)
    mask = morphology.binary_fill_holes(mask).astype(np.uint8)
    return mask

def crop_to_breast(img, mask):
    # aplica a silhueta à imagem original (remove rótulos/fundo) — conforme o artigo
    # :contentReference[oaicite:30]{index=30}
    ys, xs = np.where(mask > 0)
    if len(xs) == 0:
        return img, mask
    y0, y1 = ys.min(), ys.max()
    x0, x1 = xs.min(), xs.max()
    cropped = img[y0:y1+1, x0:x1+1]
    cropped_mask = mask[y0:y1+1, x0:x1+1]
    return cropped * cropped_mask, cropped_mask

teste = read_mias_image("MIAS")

OSError: ImageIO does not generally support reading folders. Limited support may be available via specific plugins. Specify the plugin explicitly using the `plugin` kwarg, e.g. `plugin='DICOM'`